In [1]:
# importing some useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  
import time 

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from nltk.tokenize import RegexpTokenizer  
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import pickle 

# Wczytywanie danych

In [2]:
kaggle = pd.read_csv("../Data/URLs-mixed/kaggle_labeled.csv")
kaggle.head(2) # url label

,url,label
0,diaryofagameaddict.com,bad
1,espdesign.com.au,bad


## Dane polska

In [18]:
gov = pd.read_csv("../Data/URLs-good/gov_poland.csv", skiprows=2, sep=';', usecols=[1], names=["url"])
gov.head(2)

,url
0,http://1bcz.wp.mil.pl
1,http://1bdm.wp.mil.pl/pl/28.html


In [19]:
gov["url"] = gov["url"].astype(str)
gov["label"] = "good"
gov.head(2)

,url,label
0,http://1bcz.wp.mil.pl,good
1,http://1bdm.wp.mil.pl/pl/28.html,good


## Dane cert

In [20]:
cert = pd.read_csv("https://hole.cert.pl/domains/domains.txt", names=["url"])
cert.head(2)

,url
0,008753331120.com
1,02-wiadomosci.com.pl


In [21]:
cert["url"] = cert["url"].astype(str)
cert["label"] = "bad"
cert.head(2)

,url,label
0,008753331120.com,bad
1,02-wiadomosci.com.pl,bad


In [22]:
df = pd.concat([kaggle,gov,cert])
df.head(2)

,url,label
0,diaryofagameaddict.com,bad
1,espdesign.com.au,bad


In [128]:
df["url"] = df.url.astype(str)

# Trenowanie modelu

In [130]:
def count_special_symbols(domain):
    counter = 0
    for char in domain:
        if char.isalpha() or char.isdigit():
            continue
        else:
            counter += 1
    return counter


def count_digits(domain):
    counter = 0
    for char in domain:
        if char.isdigit():
            counter += 1
    return counter

In [153]:
from urllib3.util import parse_url
def parseurl(url):
    try:
        url = url.replace("[","")
        url = url.replace("]","")
        url = parse_url(url)
        return str(url.host)
    except Exception as e:
        if "//" in url:
            url = url.split("//")[1]
        url = url[:url.find("/")]
        return str(url)

In [155]:
df["parsed_url"] = df.url.apply(parseurl)

In [164]:
good = df[df["label"] == "good"]
bad = df[df["label"] == "bad"]

In [165]:
data = pd.DataFrame(df.iloc[1:200,2])

In [166]:
data.columns = ["url"]

In [169]:
import Levenshtein

def get_best_lev_match(good_domains: pd.DataFrame,domains: pd.DataFrame):
    """
    Calculates Levenshtein distance of given string to all domains in top-100 polish database and returns best domain
    and it's score.

    Returns
    -------

    """

    known_domains = pd.DataFrame(good_domains['parsed_url'].str.split(".", n=1, expand=True)[0])
    known_domains.columns = ["url"]

    return domains.apply(lambda y: known_domains.iloc[
        known_domains.apply(lambda x: Levenshtein.jaro_winkler(y['url'], x['url'], 0.25), axis=1).idxmax()
    ], axis=1)

In [170]:
get_best_lev_match(good, data)

,url
1,people
2,sports
3,ontheradio
4,archive
5,big12sports
...,...
195,facebook
196,halloweenmovie
197,www
198,tripadvisor


In [ ]:
known_domains.apply(lambda x: Levenshtein.jaro_winkler(good.iloc[1,1], x['url'], 0.25), axis=1).idxmax()

In [151]:
known_domains["url"]

42767              01453
42768            015fb31
42769            02bee66
42770            02ec0a3
42771             032255
              ...       
69521              https
69522              https
69523              https
69524              https
69525    zup-ornontowice
Name: url, Length: 414347, dtype: object